<a href="https://colab.research.google.com/github/aimiliavaitsi/Biopliastics_Sentiment_Project/blob/main/Top_30_Words_per_Emotion_category%2C_Year%2C_Quarter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install and import necessary libraries
!pip install pandas
!pip install xlrd
!pip install nrclex
!pip install matplotlib
!pip install nltk

import nltk
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from collections import Counter
import string
import pandas as pd
from nrclex import NRCLex
from google.colab import files

# Step 2: Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Step 3: Upload Excel file (Book 1)
uploaded = files.upload()

# Step 4: Read the Excel file into a DataFrame
df = pd.read_excel(list(uploaded.keys())[0])

# Ensure the columns "Cleaned_Comment", "Year", and "Year-Quarter" exist
required_columns = ['Cleaned_Comment', 'Year', 'Year-Quarter']
if not all(column in df.columns for column in required_columns):
    raise ValueError(f"The Excel file must contain columns named {required_columns}")

# Step 5: Define stopwords and punctuation to filter out
# Include specific words you want to exclude
additional_stop_words = {
    'not', 'much', "'s", "n't", 'would', '’', 'like', 'time', 'thing', '...', 'https', 'could', "''",
    'even', "'m", 'used', "'re", '``', 'make', 'one', 'also', 'get',
    'really', 'see', 'made', 'know', 'need', 'think', 'use', 'still'
}
stop_words = set(stopwords.words('english')).union(additional_stop_words)
punctuation = set(string.punctuation)

# Step 6: Create dictionaries to store word frequencies per category
emotion_word_freq = {emotion: Counter() for emotion in ['anticipation', 'joy', 'surprise', 'trust', 'anger', 'fear', 'sadness', 'disgust']}
year_word_freq = {}
year_quarter_word_freq = {}

# Step 7: Tokenize comments and count word frequencies
for i, row in df.iterrows():
    comment = row['Cleaned_Comment']
    year = row['Year']
    year_quarter = row['Year-Quarter']

    # Ensure the comment is not empty
    if pd.isna(comment) or len(comment.strip()) == 0:
        continue

    # Tokenize the comment into words
    words = nltk.word_tokenize(comment.lower())

    # Filter out stopwords and punctuation
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]

    # Analyze the comment with NRCLex
    text_object = NRCLex(comment)
    emotions = text_object.raw_emotion_scores  # Get the raw emotion scores for the comment

    # Update word frequencies for each emotion
    for emotion, score in emotions.items():
        if emotion in emotion_word_freq:
            emotion_word_freq[emotion].update(filtered_words)

    # Update word frequencies for the year
    if year not in year_word_freq:
        year_word_freq[year] = Counter()
    year_word_freq[year].update(filtered_words)

    # Update word frequencies for the year-quarter
    if year_quarter not in year_quarter_word_freq:
        year_quarter_word_freq[year_quarter] = Counter()
    year_quarter_word_freq[year_quarter].update(filtered_words)

# Step 8: Extract top 30 words for each category
top_words_per_emotion = {emotion: freq.most_common(30) for emotion, freq in emotion_word_freq.items()}
top_words_per_year = {year: freq.most_common(30) for year, freq in year_word_freq.items()}
top_words_per_year_quarter = {yq: freq.most_common(30) for yq, freq in year_quarter_word_freq.items()}

# Step 9: Prepare Data for Excel Export

# Initialize lists to store data for the DataFrame
emotion_excel_data = []
year_excel_data = []
year_quarter_excel_data = []

# Loop through the top words for each emotion and prepare rows for the DataFrame
for emotion, top_words in top_words_per_emotion.items():
    for word, count in top_words:
        emotion_excel_data.append([emotion, word, count])

# Loop through the top words for each year and prepare rows for the DataFrame
for year, top_words in top_words_per_year.items():
    for word, count in top_words:
        year_excel_data.append([year, word, count])

# Loop through the top words for each year-quarter and prepare rows for the DataFrame
for year_quarter, top_words in top_words_per_year_quarter.items():
    for word, count in top_words:
        year_quarter_excel_data.append([year_quarter, word, count])

# Create DataFrames for each category
emotion_df = pd.DataFrame(emotion_excel_data, columns=['Emotion Category', 'Top Words', 'Count'])
year_df = pd.DataFrame(year_excel_data, columns=['Year', 'Top Words', 'Count'])
year_quarter_df = pd.DataFrame(year_quarter_excel_data, columns=['Year-Quarter', 'Top Words', 'Count'])

# Step 10: Save the results to an Excel file with multiple sheets
excel_file_path = "/content/Top_30_Words_Analysis.xlsx"
with pd.ExcelWriter(excel_file_path) as writer:
    emotion_df.to_excel(writer, sheet_name='Emotion Analysis', index=False)
    year_df.to_excel(writer, sheet_name='Year Analysis', index=False)
    year_quarter_df.to_excel(writer, sheet_name='Year-Quarter Analysis', index=False)

# Step 11: Provide a download link for the Excel file
files.download(excel_file_path)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving Book1.xlsx to Book1 (2).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>